--- Day 5: Print Queue ---

Satisfied with their search on Ceres, the squadron of scholars suggests subsequently scanning the stationery stacks of sub-basement 17.

The North Pole printing department is busier than ever this close to Christmas, and while The Historians continue their search of this historically significant facility, an Elf operating a very familiar printer beckons you over.

The Elf must recognize you, because they waste no time explaining that the new sleigh launch safety manual updates won't print correctly. Failure to update the safety manuals would be dire indeed, so you offer your services.

Safety protocols clearly indicate that new pages for the safety manuals must be printed in a very specific order. The notation X|Y means that if both page number X and page number Y are to be produced as part of an update, page number X must be printed at some point before page number Y.

The Elf has for you both the page ordering rules and the pages to produce in each update (your puzzle input), but can't figure out whether each update has the pages in the right order.

For example:

47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47

The first section specifies the page ordering rules, one per line. The first rule, 47|53, means that if an update includes both page number 47 and page number 53, then page number 47 must be printed at some point before page number 53. (47 doesn't necessarily need to be immediately before 53; other pages are allowed to be between them.)

The second section specifies the page numbers of each update. Because most safety manuals are different, the pages needed in the updates are different too. The first update, 75,47,61,53,29, means that the update consists of page numbers 75, 47, 61, 53, and 29.

To get the printers going as soon as possible, start by identifying which updates are already in the right order.

In the above example, the first update (75,47,61,53,29) is in the right order:

    75 is correctly first because there are rules that put each other page after it: 75|47, 75|61, 75|53, and 75|29.
    47 is correctly second because 75 must be before it (75|47) and every other page must be after it according to 47|61, 47|53, and 47|29.
    61 is correctly in the middle because 75 and 47 are before it (75|61 and 47|61) and 53 and 29 are after it (61|53 and 61|29).
    53 is correctly fourth because it is before page number 29 (53|29).
    29 is the only page left and so is correctly last.

Because the first update does not include some page numbers, the ordering rules involving those missing page numbers are ignored.

The second and third updates are also in the correct order according to the rules. Like the first update, they also do not include every page number, and so only some of the ordering rules apply - within each update, the ordering rules that involve missing page numbers are not used.

The fourth update, 75,97,47,61,53, is not in the correct order: it would print 75 before 97, which violates the rule 97|75.

The fifth update, 61,13,29, is also not in the correct order, since it breaks the rule 29|13.

The last update, 97,13,75,29,47, is not in the correct order due to breaking several rules.

For some reason, the Elves also need to know the middle page number of each update being printed. Because you are currently only printing the correctly-ordered updates, you will need to find the middle page number of each correctly-ordered update. In the above example, the correctly-ordered updates are:

75,47,61,53,29
97,61,53,29,13
75,29,13

These have middle page numbers of 61, 53, and 29 respectively. Adding these page numbers together gives 143.

Of course, you'll need to be careful: the actual list of page ordering rules is bigger and more complicated than the above example.

Determine which updates are already in the correct order. What do you get if you add up the middle page number from those correctly-ordered updates?


In [ ]:
class PageOrderValidator:
    def __init__(self):
        self.rules = {}  # Dictionary to store rules
        self.updates = []  # List to store updates
        
    # Dodajemy przykładowe dane jako stałą
    EXAMPLE_DATA = """47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47"""
        
    def load_from_string(self, content):
        """Load rules and updates from string content"""
        # Split content into rules and updates sections
        sections = content.strip().split('\n\n')
        if len(sections) != 2:
            raise ValueError("Input must contain rules and updates separated by empty line")
            
        # Process rules
        rules_section = sections[0].strip().split('\n')
        for rule in rules_section:
            if '|' not in rule:
                continue
            before, after = map(int, rule.split('|'))
            if before not in self.rules:
                self.rules[before] = set()
            self.rules[before].add(after)
            
        # Process updates
        updates_section = sections[1].strip().split('\n')
        for update in updates_section:
            pages = list(map(int, update.strip().split(',')))
            self.updates.append(pages)
            
    def load_from_file(self, filename):
        """Load rules and updates from file"""
        try:
            with open(filename, 'r') as file:
                content = file.read()
                self.load_from_string(content)
        except FileNotFoundError:
            print(f"Error: File {filename} not found")
            return False
        except Exception as e:
            print(f"Error loading file: {str(e)}")
            return False
        return True
    
    def validate_update(self, update):
        """Validate if an update follows all rules"""
        # Create index mapping for current update
        page_positions = {page: idx for idx, page in enumerate(update)}
        
        # Check each rule
        for before_page, after_pages in self.rules.items():
            for after_page in after_pages:
                # Skip if either page is not in the update
                if before_page not in page_positions or after_page not in page_positions:
                    continue
                    
                # Check if before_page comes before after_page
                if page_positions[before_page] >= page_positions[after_page]:
                    return False
        return True
    
    def validate_all_updates(self):
        """Validate all updates and return results"""
        results = []
        for idx, update in enumerate(self.updates, 1):
            is_valid = self.validate_update(update)
            results.append({
                'update_number': idx,
                'pages': update,
                'valid': is_valid
            })
        return results
    
    def print_validation_results(self, results):
        """Print validation results in a readable format"""
        print("\nWalidacja aktualizacji:")
        print("-" * 50)
        
        for result in results:
            status = "✓ Poprawna" if result['valid'] else "✗ Niepoprawna"
            print(f"Aktualizacja {result['update_number']}:")
            print(f"Strony: {result['pages']}")
            print(f"Status: {status}")
            print("-" * 50)
            
        valid_count = sum(1 for r in results if r['valid'])
        print(f"\nPodsumowanie:")
        print(f"Poprawnych aktualizacji: {valid_count}")
        print(f"Niepoprawnych aktualizacji: {len(results) - valid_count}")
    
    def get_middle_numbers(self, results):
        """Get middle numbers from valid updates and calculate their sum"""
        middle_numbers = []
        valid_updates = []  # Lista do przechowywania poprawnych aktualizacji
        
        for result in results:
            if result['valid']:
                pages = result['pages']
                valid_updates.append(pages)  # Zapisz poprawną aktualizację
                middle_idx = len(pages) // 2
                middle_number = pages[middle_idx]
                middle_numbers.append(middle_number)
                print(f"Aktualizacja {result['update_number']}: środkowy numer to {middle_number}")
        
        print("\nPoprawne aktualizacje:")
        for idx, update in enumerate(valid_updates, 1):
            print(f"{idx}. {update}")
        
        total_sum = sum(middle_numbers)
        print(f"\nŚrodkowe numery z poprawnych aktualizacji: {middle_numbers}")
        print(f"Suma środkowych numerów: {total_sum}")
        return total_sum, valid_updates  # Zwracamy też listę poprawnych aktualizacji


validator = PageOrderValidator()
validator.load_from_string(PageOrderValidator.EXAMPLE_DATA)

print("Załadowane reguły:")
for before_page, after_pages in validator.rules.items():
    for after_page in after_pages:
        print(f"{before_page} musi być przed {after_page}")
        
print("\nZaładowane aktualizacje:")
for update in validator.updates:
    print(update)
    
# Najpierw walidujemy wszystkie aktualizacje
results = validator.validate_all_updates()
validator.print_validation_results(results)

# Potem analizujemy środkowe numery
print("\nAnaliza środkowych numerów:")
middle_sum, valid_updates = validator.get_middle_numbers(results)

print(f"\nKońcowy wynik (suma środkowych numerów): {middle_sum}")

In [ ]:
validator = PageOrderValidator()
validator.load_from_file("input.txt")

print("Załadowane reguły:")
for before_page, after_pages in validator.rules.items():
    for after_page in after_pages:
        print(f"{before_page} musi być przed {after_page}")
        
print("\nZaładowane aktualizacje:")
for update in validator.updates:
    print(update)
    
# Najpierw walidujemy wszystkie aktualizacje
results = validator.validate_all_updates()
validator.print_validation_results(results)

# Potem analizujemy środkowe numery
print("\nAnaliza środkowych numerów:")
middle_sum, valid_updates = validator.get_middle_numbers(results)

print(f"\nKońcowy wynik (suma środkowych numerów): {middle_sum}")

our puzzle answer was 5948.

The first half of this puzzle is complete! It provides one gold star: *
--- Part Two ---

While the Elves get to work printing the correctly-ordered updates, you have a little time to fix the rest of them.

For each of the incorrectly-ordered updates, use the page ordering rules to put the page numbers in the right order. For the above example, here are the three incorrectly-ordered updates and their correct orderings:

    75,97,47,61,53 becomes 97,75,47,61,53.
    61,13,29 becomes 61,29,13.
    97,13,75,29,47 becomes 97,75,47,29,13.

After taking only the incorrectly-ordered updates and ordering them correctly, their middle page numbers are 47, 29, and 47. Adding these together produces 123.

Find the updates which are not in the correct order. What do you get if you add up the middle page numbers after correctly ordering just those updates?


In [9]:
class PageOrderValidator:
    def __init__(self):
        self.rules = {}  # Dictionary to store rules
        self.updates = []  # List to store updates
        
    # Dodajemy przykładowe dane jako stałą
    EXAMPLE_DATA = """47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47"""
        
    def load_from_string(self, content):
        """Load rules and updates from string content"""
        # Split content into rules and updates sections
        sections = content.strip().split('\n\n')
        if len(sections) != 2:
            raise ValueError("Input must contain rules and updates separated by empty line")
            
        # Process rules
        rules_section = sections[0].strip().split('\n')
        for rule in rules_section:
            if '|' not in rule:
                continue
            before, after = map(int, rule.split('|'))
            if before not in self.rules:
                self.rules[before] = set()
            self.rules[before].add(after)
            
        # Process updates
        updates_section = sections[1].strip().split('\n')
        for update in updates_section:
            pages = list(map(int, update.strip().split(',')))
            self.updates.append(pages)
            
    def load_from_file(self, filename):
        """Load rules and updates from file"""
        try:
            with open(filename, 'r') as file:
                content = file.read()
                self.load_from_string(content)
        except FileNotFoundError:
            print(f"Error: File {filename} not found")
            return False
        except Exception as e:
            print(f"Error loading file: {str(e)}")
            return False
        return True
    
    def validate_update(self, update):
        """Validate if an update follows all rules"""
        # Create index mapping for current update
        page_positions = {page: idx for idx, page in enumerate(update)}
        
        # Check each rule
        for before_page, after_pages in self.rules.items():
            for after_page in after_pages:
                # Skip if either page is not in the update
                if before_page not in page_positions or after_page not in page_positions:
                    continue
                    
                # Check if before_page comes before after_page
                if page_positions[before_page] >= page_positions[after_page]:
                    return False
        return True
    
    def validate_all_updates(self):
        """Validate all updates and return results"""
        results = []
        for idx, update in enumerate(self.updates, 1):
            is_valid = self.validate_update(update)
            results.append({
                'update_number': idx,
                'pages': update,
                'valid': is_valid
            })
        return results
    
    def print_validation_results(self, results):
        """Print validation results in a readable format"""
        print("\nWalidacja aktualizacji:")
        print("-" * 50)
        
        for result in results:
            status = "✓ Poprawna" if result['valid'] else "✗ Niepoprawna"
            print(f"Aktualizacja {result['update_number']}:")
            print(f"Strony: {result['pages']}")
            print(f"Status: {status}")
            print("-" * 50)
            
        valid_count = sum(1 for r in results if r['valid'])
        print(f"\nPodsumowanie:")
        print(f"Poprawnych aktualizacji: {valid_count}")
        print(f"Niepoprawnych aktualizacji: {len(results) - valid_count}")
    
    def get_middle_numbers(self, results):
        """Get middle numbers from valid updates and calculate their sum"""
        middle_numbers = []
        valid_updates = []  # Lista do przechowywania poprawnych aktualizacji
        
        for result in results:
            if result['valid']:
                pages = result['pages']
                valid_updates.append(pages)  # Zapisz poprawną aktualizację
                middle_idx = len(pages) // 2
                middle_number = pages[middle_idx]
                middle_numbers.append(middle_number)
                print(f"Aktualizacja {result['update_number']}: środkowy numer to {middle_number}")
        
        print("\nPoprawne aktualizacje:")
        for idx, update in enumerate(valid_updates, 1):
            print(f"{idx}. {update}")
        
        total_sum = sum(middle_numbers)
        print(f"\nŚrodkowe numery z poprawnych aktualizacji: {middle_numbers}")
        print(f"Suma środkowych numerów: {total_sum}")
        return total_sum, valid_updates  # Zwracamy też listę poprawnych aktualizacji

    def fix_update_order(self, pages):
        """Fix the order of pages according to rules"""
        n = len(pages)
        # Convert to list to allow modifications
        result = pages.copy()
        
        # Keep trying to fix the order until no more changes are needed
        while True:
            changes_made = False
            for before_page, after_pages in self.rules.items():
                for after_page in after_pages:
                    if before_page in result and after_page in result:
                        idx_before = result.index(before_page)
                        idx_after = result.index(after_page)
                        if idx_before > idx_after:
                            # Swap to fix order
                            result.remove(before_page)
                            new_pos = result.index(after_page)
                            result.insert(new_pos, before_page)
                            changes_made = True
            
            if not changes_made:
                break
                
        return result

    def analyze_updates(self, results):
        """Analyze both valid and fixed invalid updates"""
        valid_updates = []
        invalid_updates = []
        valid_middle_numbers = []
        fixed_middle_numbers = []
        
        # Process all updates
        for result in results:
            pages = result['pages']
            middle_idx = len(pages) // 2
            
            if result['valid']:
                valid_updates.append(pages)
                valid_middle_numbers.append(pages[middle_idx])
            else:
                fixed_order = self.fix_update_order(pages)
                invalid_updates.append({
                    'original': pages,
                    'fixed': fixed_order,
                    'middle': fixed_order[middle_idx]
                })
                fixed_middle_numbers.append(fixed_order[middle_idx])
        
        # Print results
        print("\nPoprawne aktualizacje:")
        for idx, update in enumerate(valid_updates, 1):
            print(f"{idx}. {update} (środkowy: {update[len(update)//2]})")
        
        print("\nNaprawione niepoprawne aktualizacje:")
        for idx, update in enumerate(invalid_updates, 1):
            print(f"{idx}. {update['original']} -> {update['fixed']} (środkowy: {update['middle']})")
        
        valid_sum = sum(valid_middle_numbers)
        fixed_sum = sum(fixed_middle_numbers)
        total_sum = valid_sum + fixed_sum
        
        print(f"\nSuma środkowych z poprawnych aktualizacji: {valid_sum}")
        print(f"Suma środkowych z naprawionych aktualizacji: {fixed_sum}")
        print(f"Całkowita suma środkowych: {total_sum}")
        
        return {
            'valid_sum': valid_sum,
            'fixed_sum': fixed_sum,
            'total_sum': total_sum,
            'valid_updates': valid_updates,
            'fixed_updates': invalid_updates
        }


validator = PageOrderValidator()
validator.load_from_string(PageOrderValidator.EXAMPLE_DATA)

print("Załadowane reguły:")
for before_page, after_pages in validator.rules.items():
    for after_page in after_pages:
        print(f"{before_page} musi być przed {after_page}")
        
print("\nZaładowane aktualizacje:")
for update in validator.updates:
    print(update)
    
# Validate and analyze updates
results = validator.validate_all_updates()
validator.print_validation_results(results)

# Analyze and fix updates
analysis = validator.analyze_updates(results)


Załadowane reguły:
47 musi być przed 29
47 musi być przed 13
47 musi być przed 61
47 musi być przed 53
97 musi być przed 75
97 musi być przed 13
97 musi być przed 47
97 musi być przed 61
97 musi być przed 53
97 musi być przed 29
75 musi być przed 13
75 musi być przed 47
75 musi być przed 29
75 musi być przed 53
75 musi być przed 61
61 musi być przed 29
61 musi być przed 53
61 musi być przed 13
29 musi być przed 13
53 musi być przed 13
53 musi być przed 29

Załadowane aktualizacje:
[75, 47, 61, 53, 29]
[97, 61, 53, 29, 13]
[75, 29, 13]
[75, 97, 47, 61, 53]
[61, 13, 29]
[97, 13, 75, 29, 47]

Walidacja aktualizacji:
--------------------------------------------------
Aktualizacja 1:
Strony: [75, 47, 61, 53, 29]
Status: ✓ Poprawna
--------------------------------------------------
Aktualizacja 2:
Strony: [97, 61, 53, 29, 13]
Status: ✓ Poprawna
--------------------------------------------------
Aktualizacja 3:
Strony: [75, 29, 13]
Status: ✓ Poprawna
------------------------------------------

In [10]:
validator = PageOrderValidator()
validator.load_from_file("input.txt")

# print("Załadowane reguły:")
# for before_page, after_pages in validator.rules.items():
#     for after_page in after_pages:
#         print(f"{before_page} musi być przed {after_page}")
        
# print("\nZaładowane aktualizacje:")
# for update in validator.updates:
#     print(update)
    
# Validate and analyze updates
results = validator.validate_all_updates()
validator.print_validation_results(results)

# Analyze and fix updates
analysis = validator.analyze_updates(results)


Walidacja aktualizacji:
--------------------------------------------------
Aktualizacja 1:
Strony: [38, 33, 35, 91, 62, 74, 57, 54, 15, 73, 64, 61, 65, 32, 25, 19, 21, 39, 58, 26, 79]
Status: ✓ Poprawna
--------------------------------------------------
Aktualizacja 2:
Strony: [46, 62, 56, 61, 35, 88, 74]
Status: ✗ Niepoprawna
--------------------------------------------------
Aktualizacja 3:
Strony: [24, 17, 55, 67, 42, 16, 58]
Status: ✗ Niepoprawna
--------------------------------------------------
Aktualizacja 4:
Strony: [16, 39, 24, 25, 28, 17, 98, 41, 64, 21, 29, 42, 13, 79, 61, 19, 14, 58, 67, 94, 32]
Status: ✗ Niepoprawna
--------------------------------------------------
Aktualizacja 5:
Strony: [34, 56, 97, 37, 38, 33, 35, 91, 62, 54, 15, 73, 64, 42, 65, 32, 17]
Status: ✓ Poprawna
--------------------------------------------------
Aktualizacja 6:
Strony: [37, 46, 38, 33, 91, 62, 74, 57, 54, 15, 73, 64, 61, 42, 65, 32, 17, 25, 19, 41, 21, 39, 58]
Status: ✓ Poprawna
------------

Your puzzle answer was 3062.

Both parts of this puzzle are complete! They provide two gold stars: **